In [2]:
import pandas as pd
import plotly.express as px
import json
import os
import plotly
from datetime import datetime

In [4]:
def load_and_clean_data(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()  # Clean column names
    df['date of sale'] = pd.to_datetime(df['date of sale'], errors='coerce')
    df['date of lead'] = pd.to_datetime(df['date of lead'], errors='coerce')
    df['total sale amount'] = pd.to_numeric(df['total sale amount'], errors='coerce')
    category_cols = ['Kitchen', 'Bathroom', 'Flooring', 'Remodel', 'Cabinet']
    df[category_cols] = df[category_cols].apply(pd.to_numeric, errors='coerce')
    return df

In [6]:
def export_plotly_json(fig, file_name, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    json_path = os.path.join(output_dir, f"{file_name}.json")
    with open(json_path, "w") as f:
        json.dump(fig, f, cls=plotly.utils.PlotlyJSONEncoder)

In [8]:
def generate_all_charts_as_json(df, output_dir="Dashboard"):
    df['year'] = df['date of sale'].dt.year
    category_cols = ['Kitchen', 'Bathroom', 'Flooring', 'Remodel', 'Cabinet']

    for year in [2024, 2025]:
        year_df = df[df['year'] == year]
        totals = year_df[category_cols].sum()
        fig = px.bar(
            x=totals.index,
            y=totals.values,
            labels={'x': 'Category', 'y': 'Total Amount'},
            title=f"Total Dollar Amount by Category ({year})"
        )
        export_plotly_json(fig, f"bar_data_{year}", output_dir)

    df['days_to_sale'] = (df['date of sale'] - df['date of lead']).dt.days
    df_scatter = df.dropna(subset=['total sale amount', 'days_to_sale', 'Last name'])
    scatter_fig = px.scatter(
        df_scatter,
        x='days_to_sale',
        y='total sale amount',
        hover_name='Last name',
        title='Days to Sale vs Total Sale Amount'
    )
    export_plotly_json(scatter_fig, "scatter_data", output_dir)

    df_week = df.dropna(subset=['date of sale', 'total sale amount'])
    df_week['week'] = df_week['date of sale'].dt.to_period('W').apply(lambda r: r.start_time)
    weekly = df_week.groupby('week')['total sale amount'].sum().reset_index()
    weekly_fig = px.line(
        weekly,
        x='week',
        y='total sale amount',
        title='Weekly Sales Totals',
        markers=True
    )
    export_plotly_json(weekly_fig, "weekly_line_data", output_dir)

In [10]:
# Run chart generation
df = load_and_clean_data("Job list - Job Break Downs.csv")
generate_all_charts_as_json(df)

C:\Users\Andre\AppData\Local\Temp\ipykernel_11528\816763714.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date of sale'] = pd.to_datetime(df['date of sale'], errors='coerce')
C:\Users\Andre\AppData\Local\Temp\ipykernel_11528\816763714.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date of lead'] = pd.to_datetime(df['date of lead'], errors='coerce')
